<a href="https://colab.research.google.com/github/ullasbc02/obesity-risk-analytics/blob/main/01_data_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extracting Datasets and merging based on FIPS code


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import pandas as pd
import os

In [2]:
RAW_DATA_PATH = '/content/drive/MyDrive/obesity-risk-analytics/data/raw'

In [5]:
# Get the directory path from the variable
directory_path = RAW_DATA_PATH

# List the contents of the directory
contents = os.listdir(directory_path)

# Print the contents
for item in contents:
    print(item)

Obesity Prevalence.csv
Poverty.csv
Leisure-Time Physical Inactivity.csv
Health Professional Shortage Areas_ Primary Care, by County.csv
Low Access to Healthy Food.csv
Median Household Income.csv
cc-est2024-alldata.csv
Unemployment Rate.csv
CC-EST2024-ALLDATA.pdf


As the dataset contents different years combined, let's separate to filter out individual year



In [8]:
obesity_AllYear_df = pd.read_csv(
    os.path.join(RAW_DATA_PATH, 'Obesity Prevalence.csv'),
    dtype={"geoId": str, "GEOID": str, "county_fips": str}
)

poverty_AllYear_df = pd.read_csv(
    os.path.join(RAW_DATA_PATH, 'Poverty.csv'),
    dtype={"geoId": str, "GEOID": str, "county_fips": str}
)

leisure_time_physical_inactivity_AllYear_df = pd.read_csv(
    os.path.join(RAW_DATA_PATH, 'Leisure-Time Physical Inactivity.csv'),
    dtype={"geoId": str, "GEOID": str, "county_fips": str}
)

health_professional_shortage_areas_AllYear_df = pd.read_csv(
    os.path.join(RAW_DATA_PATH, 'Health Professional Shortage Areas_ Primary Care, by County.csv'),
    dtype={"geoId": str, "GEOID": str, "county_fips": str}
)

low_access_health_food_AllYear_df = pd.read_csv(
    os.path.join(RAW_DATA_PATH, 'Low Access to Healthy Food.csv'),
    dtype={"geoId": str, "GEOID": str, "county_fips": str}
)

median_household_income_AllYear_df = pd.read_csv(
    os.path.join(RAW_DATA_PATH, 'Median Household Income.csv'),
    dtype={"geoId": str, "GEOID": str, "county_fips": str}
)

unemployment_rate_AllYear_df = pd.read_csv(
    os.path.join(RAW_DATA_PATH, 'Unemployment Rate.csv'),
    dtype={"geoId": str, "GEOID": str, "county_fips": str}
)

In [9]:
def clean_columns(df):
    df.columns = (
        df.columns.str.lower()
        .str.strip()
        .str.replace(" ", "_")
        .str.replace("-", "_")
    )
    return df

datasets = [
    obesity_AllYear_df, poverty_AllYear_df,
    leisure_time_physical_inactivity_AllYear_df,
    health_professional_shortage_areas_AllYear_df,
    low_access_health_food_AllYear_df,
    median_household_income_AllYear_df,
    unemployment_rate_AllYear_df
]

datasets = [clean_columns(df) for df in datasets]

In [11]:
for name, df in [
    ("obesity", obesity_AllYear_df),
    ("poverty", poverty_AllYear_df),
    ("physical_inactivity", leisure_time_physical_inactivity_AllYear_df),
    ("low_food_access", low_access_health_food_AllYear_df),
    ("median_income", median_household_income_AllYear_df),
    ("unemployment", unemployment_rate_AllYear_df),
    ("shortage", health_professional_shortage_areas_AllYear_df),
]:
    print(name, df["geoid"].duplicated().sum())


obesity 53311
poverty 43973
physical_inactivity 53308
low_food_access 0
median_income 40828
unemployment 40827
shortage 0


In [12]:
# Filter each dataframe for the year 2019
obesity_2019_df = obesity_AllYear_df[obesity_AllYear_df['year'] == 2019].copy()
poverty_2019_df = poverty_AllYear_df[poverty_AllYear_df['year'] == 2019].copy()
leisure_time_physical_inactivity_2019_df = leisure_time_physical_inactivity_AllYear_df[leisure_time_physical_inactivity_AllYear_df['year'] == 2019].copy()
health_professional_shortage_areas_2025_df = health_professional_shortage_areas_AllYear_df[health_professional_shortage_areas_AllYear_df['year'] == 2025].copy()
low_access_health_food_2019_df = low_access_health_food_AllYear_df[low_access_health_food_AllYear_df['year'] == 2019].copy()
median_household_income_2019_df = median_household_income_AllYear_df[median_household_income_AllYear_df['year'] == 2019].copy()
unemployment_rate_2019_df = unemployment_rate_AllYear_df[unemployment_rate_AllYear_df['year'] == 2019].copy()

print("Filtered dataframes for the year 2019 created.")

Filtered dataframes for the year 2019 created.


In [13]:
# Apply column normalization to year-filtered datasets
filtered_datasets = [
    obesity_2019_df,
    poverty_2019_df,
    leisure_time_physical_inactivity_2019_df,
    low_access_health_food_2019_df,
    median_household_income_2019_df,
    unemployment_rate_2019_df,
    health_professional_shortage_areas_2025_df
]

filtered_datasets = [clean_columns(df) for df in filtered_datasets]

In [14]:
poverty_AllYear_df.head()

,metro_nonmetro,year,county_name,state_abbr,geoid,value
0,Metropolitan,2009,Fairfield County,CT,us-ct-001,0.082
1,Metropolitan,2009,Hartford County,CT,us-ct-003,0.101
2,Metropolitan,2009,Middlesex County,CT,us-ct-007,0.057
3,Metropolitan,2009,New Haven County,CT,us-ct-009,0.119
4,Metropolitan,2009,New London County,CT,us-ct-011,0.079


In [16]:
def normalize_geoid(df):
    # Find any column that looks like GEOID / FIPS
    geoid_col = None
    for col in df.columns:
        if col.lower() in ["geoid", "fips", "fips_code", "countyfips", "county_fips"]:
            geoid_col = col
            break

    if geoid_col is None:
        raise ValueError("No GEOID/FIPS column found in dataframe.")

    # Standardize to 5-digit string
    df["GEOID"] = df[geoid_col].astype(str).str.zfill(5)
    return df


In [17]:
obesity_2019_df = normalize_geoid(obesity_2019_df)
poverty_2019_df = normalize_geoid(poverty_2019_df)
leisure_time_physical_inactivity_2019_df = normalize_geoid(leisure_time_physical_inactivity_2019_df)
health_professional_shortage_areas_2025_df = normalize_geoid(health_professional_shortage_areas_2025_df)
low_access_health_food_2019_df = normalize_geoid(low_access_health_food_2019_df)
median_household_income_2019_df = normalize_geoid(median_household_income_2019_df)
unemployment_rate_2019_df = normalize_geoid(unemployment_rate_2019_df)


In [18]:
obesity_2019_df = obesity_2019_df[["GEOID", "value"]].rename(columns={"value":"obesity_rate"})
poverty_2019_df = poverty_2019_df[["GEOID", "value"]].rename(columns={"value":"poverty_rate"})
leisure_time_physical_inactivity_2019_df = leisure_time_physical_inactivity_2019_df[["GEOID", "value"]].rename(columns={"value":"physical_inactivity_rate"})
health_professional_shortage_areas_2025_df = health_professional_shortage_areas_2025_df[["GEOID", "value"]].rename(columns={"value":"shortage_status"})
low_access_health_food_2019_df = low_access_health_food_2019_df[["GEOID", "value"]].rename(columns={"value":"low_access_food_rate"})
median_household_income_2019_df = median_household_income_2019_df[["GEOID", "value"]].rename(columns={"value":"median_household_income"})
unemployment_rate_2019_df = unemployment_rate_2019_df[["GEOID", "value"]].rename(columns={"value":"unemployment_rate"})


In [19]:
df_master = obesity_2019_df.copy()

dfs_to_merge = [
    poverty_2019_df,
    leisure_time_physical_inactivity_2019_df,
    low_access_health_food_2019_df,
    median_household_income_2019_df,
    unemployment_rate_2019_df,
    health_professional_shortage_areas_2025_df
]

for d in dfs_to_merge:
    df_master = df_master.merge(d, on="GEOID", how="left")
    df_master.isna().sum().sort_values()


In [20]:
numeric_cols = df_master.select_dtypes(include=["float64", "int64"]).columns
df_master[numeric_cols].describe()


,obesity_rate,poverty_rate,physical_inactivity_rate,low_access_food_rate,median_household_income,unemployment_rate
count,3121.000000,3120.000000,3119.000000,3121.000000,3120.000000,3120.000000
mean,0.287710,0.144929,0.241952,0.461953,55515.232372,0.039578
std,0.048485,0.057964,0.040577,0.286982,14258.489843,0.014789
min,0.138000,0.027000,0.105000,0.000000,24732.000000,0.008000
25%,0.248002,0.104016,0.213001,0.265225,46181.000000,0.029873
50%,0.285016,0.134014,0.238011,0.452504,53241.500000,0.036828
75%,0.323002,0.175004,0.267004,0.626227,61791.000000,0.045721
max,0.453001,0.477058,0.387003,1.000000,151806.000000,0.207356


In [21]:
df_master = df_master.copy()

# Identify numeric columns only
numeric_cols = df_master.select_dtypes(include=['float64', 'int64']).columns

# Fill missing values for numeric columns only
df_master[numeric_cols] = df_master[numeric_cols].fillna(df_master[numeric_cols].median())


In [22]:
df_master.isna().mean()
df_master.isna().sum()


,0
GEOID,0
obesity_rate,0
poverty_rate,0
physical_inactivity_rate,0
low_access_food_rate,0
median_household_income,0
unemployment_rate,0
shortage_status,0


In [23]:
df_master["shortage_status"].unique()

array(['At least part of county is shortage area',
       'Whole county is shortage area', 'None of county is shortage area'],
      dtype=object)

In [24]:
shortage_map = {
    "None of county is shortage area": 0,
    "At least part of county is shortage area": 1,
    "Whole county is shortage area": 2
}

df_master["shortage_code"] = df_master["shortage_status"].map(shortage_map)

In [26]:
df_master.to_csv("/content/drive/MyDrive/obesity-risk-analytics/data/processed/merged_features.csv", index=False)

# Load Demographics Dataset

In [27]:
cc_dataset_allYear = pd.read_csv(os.path.join(RAW_DATA_PATH, 'cc-est2024-alldata.csv'), encoding='latin1')

In [28]:
cc_filtered = cc_dataset_allYear[
    (cc_dataset_allYear["YEAR"] == 1) &    # 4/1/2020 base population
    (cc_dataset_allYear["AGEGRP"] == 0)    # Total population
].copy()

# Create GEOID (STATE + COUNTY)
cc_filtered["STATE"] = cc_filtered["STATE"].astype(str).str.zfill(2)
cc_filtered["COUNTY"] = cc_filtered["COUNTY"].astype(str).str.zfill(3)
cc_filtered["GEOID"] = cc_filtered["STATE"] + cc_filtered["COUNTY"]

# Keep only needed columns
cc_clean = cc_filtered[["GEOID", "STNAME", "CTYNAME", "TOT_POP"]]
cc_clean.head()

,GEOID,STNAME,CTYNAME,TOT_POP
0,01001,Alabama,Autauga County,58800
114,01003,Alabama,Baldwin County,231767
228,01005,Alabama,Barbour County,25226
342,01007,Alabama,Bibb County,22284
456,01009,Alabama,Blount County,59130


In [29]:
cc_filtered.to_csv("/content/drive/MyDrive/obesity-risk-analytics/data/processed_final/cc_filtered.csv", index=False)

# Convert FIPS to GEOID

In [30]:
state_fips = {
    "al":"01","ak":"02","az":"04","ar":"05","ca":"06","co":"08","ct":"09","de":"10",
    "fl":"12","ga":"13","hi":"15","id":"16","il":"17","in":"18","ia":"19",
    "ks":"20","ky":"21","la":"22","me":"23","md":"24","ma":"25","mi":"26",
    "mn":"27","ms":"28","mo":"29","mt":"30","ne":"31","nv":"32","nh":"33",
    "nj":"34","nm":"35","ny":"36","nc":"37","nd":"38","oh":"39","ok":"40",
    "or":"41","pa":"42","ri":"44","sc":"45","sd":"46","tn":"47","tx":"48","ut":"49",
    "vt":"50","va":"51","wa":"53","wv":"54","wi":"55","wy":"56","dc":"11"
}

In [31]:
def convert_us_format_to_geoid(x):
    # Expected format: "us-al-003"
    parts = x.split("-")
    state_abbr = parts[1]  # "al"
    county_code = parts[2] # "003"
    state_code = state_fips[state_abbr.lower()]
    return state_code + county_code


In [32]:
df_master["GEOID_clean"] = df_master["GEOID"].apply(convert_us_format_to_geoid)

In [34]:
df_final = df_master.merge(
    cc_filtered,  # cc_filtered demographic table
    left_on="GEOID_clean",
    right_on="GEOID",
    how="left"
)

# Handle Missing values

In [36]:
df_final["GEOID_y"].isna().sum()

np.int64(9)

In [37]:
# Find all unmatched rows (GEOID_y is NaN)
unmatched_df = df_final[df_final["GEOID_y"].isna()][["GEOID_x", "GEOID_clean"]]

print("Total unmatched counties:", unmatched_df.shape[0])
unmatched_df


Total unmatched counties: 9


,GEOID_x,GEOID_clean
0,us-ct-001,09001
1,us-ct-003,09003
2,us-ct-007,09007
3,us-ct-009,09009
4,us-ct-011,09011
5,us-ct-013,09013
6,us-ct-015,09015
1159,us-ct-005,09005
1182,us-ak-261,02261


In [38]:
# List all demographic count columns from CC-EST (80+ columns)
demo_columns = df_final.columns[df_final.columns.str.contains("_MALE|_FEMALE|TOT_")]

# Fill missing demographic values with median for each column
for col in demo_columns:
    df_final[col] = df_final[col].fillna(df_final[col].median())

# If you computed pct_xx columns, fill them too
pct_cols = df_final.columns[df_final.columns.str.startswith("pct_")]
for col in pct_cols:
    df_final[col] = df_final[col].fillna(df_final[col].median())


In [39]:
df_final["GEOID_y"] = df_final["GEOID_y"].fillna(df_final["GEOID_clean"])


In [40]:
valid_state_fips = {
    "01","02","04","05","06","08","09","10","11","12","13","15","16","17","18","19",
    "20","21","22","23","24","25","26","27","28","29","30","31","32","33","34","35",
    "36","37","38","39","40","41","42","44","45","46","47","48","49","50","51","53",
    "54","55","56"
}

df_final["state_code"] = df_final["GEOID_y"].str[:2]

invalid_states = set(df_final["state_code"]) - valid_state_fips
invalid_states


set()

In [41]:
df_final["county_code"] = df_final["GEOID_y"].str[2:]
df_final[~df_final["county_code"].str.isnumeric()]


,GEOID_x,obesity_rate,poverty_rate,physical_inactivity_rate,low_access_food_rate,median_household_income,unemployment_rate,shortage_status,shortage_code,GEOID_clean,...,HBAC_FEMALE,HIAC_MALE,HIAC_FEMALE,HAAC_MALE,HAAC_FEMALE,HNAC_MALE,HNAC_FEMALE,GEOID_y,state_code,county_code


In [42]:
official_geoid = set(cc_filtered["GEOID"])
merged_geoid   = set(df_final["GEOID_y"])

missing = merged_geoid - official_geoid
missing


{'02261',
 '09001',
 '09003',
 '09005',
 '09007',
 '09009',
 '09011',
 '09013',
 '09015'}

In [43]:
exception_geoid = {
    "09001","09003","09005","09007","09009","09011","09013","09015",
    "02261"
}


In [44]:
missing_corrected = (merged_geoid - official_geoid) - exception_geoid
missing_corrected


set()

In [45]:
df_final = df_final.rename(columns={"GEOID_y": "GEOID"})
df_final = df_final.drop(columns=["GEOID_x", "GEOID_clean"], errors="ignore")


In [46]:
df_final["GEOID"] = df_final["GEOID"].astype(str).str.zfill(5)

In [47]:
rate_cols = [
    "obesity_rate","poverty_rate","physical_inactivity_rate",
    "low_access_food_rate","unemployment_rate"
]

for col in rate_cols:
    print(col, df_master[col].min(), df_master[col].max())


obesity_rate 0.13800007757547 0.45300060881659
poverty_rate 0.02700002806072 0.47705753823744
physical_inactivity_rate 0.10500005922837 0.38700297111069
low_access_food_rate 0.0 1.0
unemployment_rate 0.008 0.20735604051941


In [48]:
int_cols = df_final.columns[df_final.columns.str.contains("_MALE|_FEMALE|TOT_")]
for col in int_cols:
    # Round to the nearest whole number and then convert to pandas nullable integer type
    df_final[col] = df_final[col].round(0).astype("Int64")

# Export cleaned file

In [49]:
df_final.to_csv("/content/drive/MyDrive/obesity-risk-analytics/data/processed_final/df_final.csv", index=False)